<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_YOLO_Evaluador_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# -*- coding: utf-8 -*-
"""
Sistema de Evaluación YOLOv8-Segmentation
==============================================================
Framework completo con análisis avanzado equivalente a Mask2Former

Características principales:
- Evaluación de todos los modelos YOLOv8-seg (n, s, m, l, x)
- Máscaras raw en archivos NPZ separados (comprimidos)
- JSON con TODAS las estadísticas y análisis de Mask2Former
- Análisis geométrico completo con Shapely
- Características de textura con Mahotas y Scikit-image
- Análisis de color avanzado
- Métricas de forma y complejidad
- Exportación dual: JSON propio + formato COCO

Autor: Jesús L.
Proyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación
Universidad: Universidad Oberta de Cataluña
Fecha: 2025
"""

'\nSistema de Evaluación YOLOv8-Segmentation\n==============================================================\nFramework completo con análisis avanzado equivalente a Mask2Former\n\nCaracterísticas principales:\n- Evaluación de todos los modelos YOLOv8-seg (n, s, m, l, x)\n- Máscaras raw en archivos NPZ separados (comprimidos)\n- JSON con TODAS las estadísticas y análisis de Mask2Former\n- Análisis geométrico completo con Shapely\n- Características de textura con Mahotas y Scikit-image\n- Análisis de color avanzado\n- Métricas de forma y complejidad\n- Exportación dual: JSON propio + formato COCO\n\nAutor: Jesús L.\nProyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación\nUniversidad: Universidad Oberta de Cataluña\nFecha: 2025\n'

In [7]:
!pip install -q ultralytics
!pip install -q shapely
!pip install -q scikit-image
!pip install -q mahotas
!pip install -q opencv-python
!pip install -q scipy
!pip install -q Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 34.7 MB/s eta 0:00:00


In [22]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# =============================================================================
# IMPORTS Y CONFIGURACIÓN INICIAL
# =============================================================================

import os
import json
import logging
import hashlib
import time
import warnings
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Tuple, Optional, Any
from dataclasses import dataclass, field, asdict

import numpy as np
import torch
from PIL import Image
from tqdm import tqdm

# Ultralytics YOLOv8
from ultralytics import YOLO

# Computer Vision
import cv2

# Análisis geométrico y matemático
from shapely.geometry import Polygon, MultiPolygon
from shapely.validation import make_valid
from shapely.ops import unary_union
from scipy import ndimage
from scipy.spatial import ConvexHull, distance
from scipy.stats import entropy
from skimage import measure, morphology, filters
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import regionprops

# Análisis de textura
import mahotas
import mahotas.features

warnings.filterwarnings('ignore')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [9]:
# =============================================================================
# CONFIGURACIÓN DE MODELOS YOLOV8-SEG
# =============================================================================

@dataclass
class ModeloYOLOInfo:
    """Información de modelo YOLOv8-seg"""
    nombre_modelo: str
    nombre_corto: str
    tamaño: str
    parametros: str
    descripcion: str

    def obtener_nombre_sanitizado(self) -> str:
        tamaño_map = {'nano': 'n', 'small': 's', 'medium': 'm', 'large': 'l', 'xlarge': 'x'}
        codigo = tamaño_map.get(self.tamaño, self.tamaño[0])
        return f"yv8{codigo}_seg"

@dataclass
class ConfiguracionUmbrales:
    """Configuración de umbrales de confianza"""
    nombre: str
    valores: List[float]
    descripcion: str

    def obtener_nombre_sanitizado(self) -> str:
        return f"cfg_{self.nombre[:4]}"

In [20]:
# =============================================================================
# CONFIGURACIÓN PRINCIPAL
# =============================================================================

@dataclass
class ConfigEvaluacionYOLO:
    """Configuración principal del sistema de evaluación YOLOv8"""

    # Rutas
    DATASET_PATH: Path = Path("/content/drive/MyDrive/TFM/yolov8_seg/imagenes")
    RESULTADOS_BASE: Path = Path("/content/drive/MyDrive/TFM/yolov8_seg/resultados")

    # Parámetros de procesamiento
    MAX_IMAGENES_LOTE: int = 100
    BATCH_SIZE: int = 1
    DEVICE: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    IMG_SIZE: int = 640

    # Opciones
    GENERAR_VISUALIZACIONES: bool = True
    LIMPIAR_CACHE_CADA: int = 25
    GENERAR_FORMATO_COCO: bool = True
    USAR_NOMBRES_SANITIZADOS: bool = True

    MODELOS: List[ModeloYOLOInfo] = None
    UMBRALES: Dict[str, ConfiguracionUmbrales] = None

    def __post_init__(self):
        if self.MODELOS is None:
            self.MODELOS = [
                ModeloYOLOInfo("yolov8n-seg.pt", "yolov8n-seg", "nano", "3.4M", "Más rápido"),
                ModeloYOLOInfo("yolov8s-seg.pt", "yolov8s-seg", "small", "11.8M", "Balance"),
                ModeloYOLOInfo("yolov8m-seg.pt", "yolov8m-seg", "medium", "27.3M", "Precisión mejorada"),
                ModeloYOLOInfo("yolov8l-seg.pt", "yolov8l-seg", "large", "46.0M", "Alta precisión"),
                ModeloYOLOInfo("yolov8x-seg.pt", "yolov8x-seg", "xlarge", "71.8M", "Máxima precisión")
            ]

        if self.UMBRALES is None:
            self.UMBRALES = {
                'ultra_sensible': ConfiguracionUmbrales('ultra_sensible', [0.01, 0.05, 0.1, 0.15], 'Máxima sensibilidad'),
                'alta_sensibilidad': ConfiguracionUmbrales('alta_sensibilidad', [0.1, 0.2, 0.3, 0.4], 'Alta sensibilidad'),
                'balanceada': ConfiguracionUmbrales('balanceada', [0.25, 0.5, 0.7], 'Balance'),
                'conservadora': ConfiguracionUmbrales('conservadora', [0.5, 0.7, 0.85], 'Conservadora')
            }

    def crear_directorio_ejecucion(self) -> Path:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        directorio = self.RESULTADOS_BASE / f"ejecucion_{timestamp}"
        (directorio / "logs").mkdir(parents=True, exist_ok=True)
        (directorio / "visualizaciones").mkdir(exist_ok=True)
        (directorio / "mascaras_raw").mkdir(exist_ok=True)
        (directorio / "formato_coco").mkdir(exist_ok=True)
        return directorio

    def validar_configuracion(self) -> bool:
        if not self.DATASET_PATH.exists():
            print(f"❌ ERROR: Dataset no encontrado en {self.DATASET_PATH}")
            return False
        if not self.MODELOS or not self.UMBRALES:
            print("❌ ERROR: Configuración incompleta")
            return False
        return True

    def obtener_resumen_configuracion(self) -> Dict[str, Any]:
        return {
            'total_combinaciones': len(self.MODELOS) * len(self.UMBRALES),
            'num_modelos': len(self.MODELOS),
            'num_configs_umbral': len(self.UMBRALES),
            'dataset_path': str(self.DATASET_PATH),
            'device': self.DEVICE,
            'batch_size': self.BATCH_SIZE,
            'img_size': self.IMG_SIZE,
            'max_imagenes_lote': self.MAX_IMAGENES_LOTE
        }

In [11]:
# =============================================================================
# EXTRACTOR DE CARACTERÍSTICAS AVANZADO
# =============================================================================

class ExtractorCaracteristicasAvanzado:
    """Extractor completo de características - Compatible con Mask2Former"""

    def __init__(self, logger: logging.Logger = None):
        self.logger = logger

    def extraer_todas_caracteristicas(self, imagen: np.ndarray, mascara: np.ndarray) -> Dict[str, Any]:
        """Extrae TODAS las características como Mask2Former"""
        caracteristicas = {}

        try:
            # Geometría básica
            caracteristicas['geometria_basica'] = self._extraer_geometria_basica(mascara)

            # Características geométricas avanzadas
            caracteristicas['geometria_avanzada'] = self._extraer_geometria_avanzada(mascara)

            # Análisis de forma
            caracteristicas['forma'] = self._analizar_forma(mascara)

            # Características de textura
            caracteristicas['textura'] = self._extraer_texturas(imagen, mascara)

            # Análisis de color
            caracteristicas['color'] = self._analizar_color(imagen, mascara)

            # Momentos de Hu
            caracteristicas['momentos_hu'] = self._calcular_momentos_hu(mascara)

            # Características de contorno
            caracteristicas['contorno'] = self._analizar_contorno(mascara)

            # Complejidad
            caracteristicas['complejidad'] = self._calcular_complejidad(mascara)

            return caracteristicas

        except Exception as e:
            if self.logger:
                self.logger.error(f"Error extrayendo características: {str(e)}")
            return {}

    def _extraer_geometria_basica(self, mascara: np.ndarray) -> Dict[str, Any]:
        """Geometría básica: área, perímetro, bbox"""
        try:
            mascara_bin = (mascara > 0.5).astype(np.uint8)
            area_pixels = np.sum(mascara_bin)

            contornos, _ = cv2.findContours(mascara_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if len(contornos) == 0:
                return {'area_pixels': 0, 'perimetro': 0}

            contorno_principal = max(contornos, key=cv2.contourArea)
            perimetro = cv2.arcLength(contorno_principal, True)
            x, y, w, h = cv2.boundingRect(contorno_principal)

            return {
                'area_pixels': int(area_pixels),
                'perimetro': float(perimetro),
                'bbox': {'x': int(x), 'y': int(y), 'width': int(w), 'height': int(h)},
                'aspect_ratio': float(w / h) if h > 0 else 0,
                'extent': float(area_pixels / (w * h)) if (w * h) > 0 else 0
            }
        except Exception as e:
            return {}

    def _extraer_geometria_avanzada(self, mascara: np.ndarray) -> Dict[str, Any]:
        """Geometría avanzada con Shapely"""
        try:
            mascara_bin = (mascara > 0.5).astype(np.uint8)
            contornos, _ = cv2.findContours(mascara_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if len(contornos) == 0:
                return {}

            contorno = max(contornos, key=cv2.contourArea)
            coords = contorno.squeeze()

            if len(coords.shape) == 1 or len(coords) < 3:
                return {}

            polygon = Polygon(coords)
            if not polygon.is_valid:
                polygon = make_valid(polygon)

            # Convex hull
            hull = cv2.convexHull(contorno)
            hull_area = cv2.contourArea(hull)

            return {
                'area_shapely': float(polygon.area),
                'perimetro_shapely': float(polygon.length),
                'convexity': float(polygon.area / hull_area) if hull_area > 0 else 0,
                'solidity': float(cv2.contourArea(contorno) / hull_area) if hull_area > 0 else 0,
                'is_valid': bool(polygon.is_valid),
                'is_simple': bool(polygon.is_simple),
                'centroid': {'x': float(polygon.centroid.x), 'y': float(polygon.centroid.y)} if polygon.centroid else None
            }
        except Exception as e:
            return {}

    def _analizar_forma(self, mascara: np.ndarray) -> Dict[str, Any]:
        """Análisis de forma y compacidad"""
        try:
            mascara_bin = (mascara > 0.5).astype(np.uint8)
            contornos, _ = cv2.findContours(mascara_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if len(contornos) == 0:
                return {}

            contorno = max(contornos, key=cv2.contourArea)
            area = cv2.contourArea(contorno)
            perimetro = cv2.arcLength(contorno, True)

            # Compacidad (circularidad)
            compacidad = (4 * np.pi * area) / (perimetro ** 2) if perimetro > 0 else 0

            # Elipse ajustada
            if len(contorno) >= 5:
                ellipse = cv2.fitEllipse(contorno)
                _, (MA, ma), _ = ellipse
                eccentricity = np.sqrt(1 - (min(MA, ma) / max(MA, ma)) ** 2) if max(MA, ma) > 0 else 0
            else:
                eccentricity = 0

            # Rectángulo mínimo
            rect = cv2.minAreaRect(contorno)
            rect_area = rect[1][0] * rect[1][1]
            rectangularity = area / rect_area if rect_area > 0 else 0

            return {
                'compacidad': float(compacidad),
                'circularidad': float(compacidad),
                'excentricidad': float(eccentricity),
                'rectangularidad': float(rectangularity)
            }
        except Exception as e:
            return {}

    def _extraer_texturas(self, imagen: np.ndarray, mascara: np.ndarray) -> Dict[str, Any]:
        """Características de textura con Mahotas y Scikit-image"""
        try:
            # Convertir a escala de grises
            if len(imagen.shape) == 3:
                img_gray = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)
            else:
                img_gray = imagen

            mascara_bin = (mascara > 0.5).astype(bool)

            # Región de interés
            roi = img_gray.copy()
            roi[~mascara_bin] = 0

            # Características Haralick (Mahotas)
            try:
                haralick_features = mahotas.features.haralick(roi.astype(np.uint8), ignore_zeros=True)
                haralick_mean = np.mean(haralick_features, axis=0)

                haralick_dict = {
                    'haralick_contrast': float(haralick_mean[1]),
                    'haralick_correlation': float(haralick_mean[2]),
                    'haralick_energy': float(haralick_mean[8]),
                    'haralick_homogeneity': float(haralick_mean[4])
                }
            except:
                haralick_dict = {}

            # GLCM con scikit-image
            try:
                glcm = graycomatrix(roi.astype(np.uint8), [1], [0, np.pi/4, np.pi/2, 3*np.pi/4],
                                   levels=256, symmetric=True, normed=True)

                glcm_dict = {
                    'glcm_contrast': float(np.mean(graycoprops(glcm, 'contrast'))),
                    'glcm_dissimilarity': float(np.mean(graycoprops(glcm, 'dissimilarity'))),
                    'glcm_homogeneity': float(np.mean(graycoprops(glcm, 'homogeneity'))),
                    'glcm_energy': float(np.mean(graycoprops(glcm, 'energy'))),
                    'glcm_correlation': float(np.mean(graycoprops(glcm, 'correlation')))
                }
            except:
                glcm_dict = {}

            # Textura de Zernike (Mahotas)
            try:
                zernike = mahotas.features.zernike_moments(roi.astype(np.uint8), radius=min(roi.shape)//2)
                zernike_dict = {'zernike_mean': float(np.mean(zernike)), 'zernike_std': float(np.std(zernike))}
            except:
                zernike_dict = {}

            return {**haralick_dict, **glcm_dict, **zernike_dict}

        except Exception as e:
            return {}

    def _analizar_color(self, imagen: np.ndarray, mascara: np.ndarray) -> Dict[str, Any]:
        """Análisis completo de color"""
        try:
            mascara_bin = (mascara > 0.5).astype(bool)

            # RGB
            if len(imagen.shape) == 3 and imagen.shape[2] == 3:
                r_mean = float(np.mean(imagen[:,:,0][mascara_bin]))
                g_mean = float(np.mean(imagen[:,:,1][mascara_bin]))
                b_mean = float(np.mean(imagen[:,:,2][mascara_bin]))
                r_std = float(np.std(imagen[:,:,0][mascara_bin]))
                g_std = float(np.std(imagen[:,:,1][mascara_bin]))
                b_std = float(np.std(imagen[:,:,2][mascara_bin]))
            else:
                r_mean = g_mean = b_mean = 0.0
                r_std = g_std = b_std = 0.0

            # HSV
            try:
                img_hsv = cv2.cvtColor(imagen, cv2.COLOR_RGB2HSV)
                h_mean = float(np.mean(img_hsv[:,:,0][mascara_bin]))
                s_mean = float(np.mean(img_hsv[:,:,1][mascara_bin]))
                v_mean = float(np.mean(img_hsv[:,:,2][mascara_bin]))
            except:
                h_mean = s_mean = v_mean = 0.0

            return {
                'rgb_mean': {'r': r_mean, 'g': g_mean, 'b': b_mean},
                'rgb_std': {'r': r_std, 'g': g_std, 'b': b_std},
                'hsv_mean': {'h': h_mean, 's': s_mean, 'v': v_mean}
            }
        except Exception as e:
            return {}

    def _calcular_momentos_hu(self, mascara: np.ndarray) -> Dict[str, float]:
        """Momentos de Hu para invarianza de forma"""
        try:
            mascara_bin = (mascara > 0.5).astype(np.uint8)
            moments = cv2.moments(mascara_bin)
            hu_moments = cv2.HuMoments(moments).flatten()

            # Log para escala manejable
            hu_log = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)

            return {f'hu_{i+1}': float(hu_log[i]) for i in range(7)}
        except Exception as e:
            return {}

    def _analizar_contorno(self, mascara: np.ndarray) -> Dict[str, Any]:
        """Análisis del contorno"""
        try:
            mascara_bin = (mascara > 0.5).astype(np.uint8)
            contornos, _ = cv2.findContours(mascara_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if len(contornos) == 0:
                return {}

            contorno = max(contornos, key=cv2.contourArea)

            # Aproximación poligonal
            epsilon = 0.01 * cv2.arcLength(contorno, True)
            approx = cv2.approxPolyDP(contorno, epsilon, True)

            return {
                'num_vertices': int(len(approx)),
                'num_puntos_contorno': int(len(contorno)),
                'suavidad': float(len(approx) / len(contorno)) if len(contorno) > 0 else 0
            }
        except Exception as e:
            return {}

    def _calcular_complejidad(self, mascara: np.ndarray) -> Dict[str, Any]:
        """Métricas de complejidad de la máscara"""
        try:
            mascara_bin = (mascara > 0.5).astype(np.uint8)

            # Dimensión fractal (box-counting simplificado)
            def fractal_dimension(Z, threshold=0.9):
                assert(len(Z.shape) == 2)
                def boxcount(Z, k):
                    S = np.add.reduceat(
                        np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                        np.arange(0, Z.shape[1], k), axis=1)
                    return len(np.where((S > 0) & (S < k*k))[0])

                Z = (Z < threshold)
                p = min(Z.shape)
                n = 2**np.floor(np.log(p)/np.log(2))
                n = int(np.log(n)/np.log(2))
                sizes = 2**np.arange(n, 1, -1)
                counts = []
                for size in sizes:
                    counts.append(boxcount(Z, size))

                coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
                return -coeffs[0]

            try:
                dim_fractal = fractal_dimension(mascara_bin.astype(float))
            except:
                dim_fractal = 0.0

            # Entropía
            hist, _ = np.histogram(mascara[mascara > 0], bins=256, range=(0, 1))
            hist = hist / hist.sum()
            entropia = entropy(hist + 1e-10)

            return {
                'dimension_fractal': float(dim_fractal),
                'entropia': float(entropia)
            }
        except Exception as e:
            return {}

In [12]:
# =============================================================================
# ANALIZADOR DE MÁSCARAS
# =============================================================================

class MaskAnalyzer:
    """Análisis especializado de máscaras - Compatible con Mask2Former"""

    def __init__(self, logger: logging.Logger = None):
        self.logger = logger

    def analizar_mascara_completa(self, mascara: np.ndarray, imagen: np.ndarray) -> Dict[str, Any]:
        """Análisis completo de una máscara"""
        try:
            return {
                'estadisticas_basicas': self._estadisticas_basicas(mascara),
                'distribucion_valores': self._analizar_distribucion(mascara),
                'calidad_mascara': self._evaluar_calidad(mascara),
                'fragmentacion': self._analizar_fragmentacion(mascara)
            }
        except Exception as e:
            if self.logger:
                self.logger.error(f"Error en análisis de máscara: {str(e)}")
            return {}

    def _estadisticas_basicas(self, mascara: np.ndarray) -> Dict[str, float]:
        """Estadísticas básicas de la máscara"""
        return {
            'mean': float(np.mean(mascara)),
            'std': float(np.std(mascara)),
            'min': float(np.min(mascara)),
            'max': float(np.max(mascara)),
            'median': float(np.median(mascara))
        }

    def _analizar_distribucion(self, mascara: np.ndarray) -> Dict[str, Any]:
        """Distribución de valores en la máscara"""
        hist, bins = np.histogram(mascara.flatten(), bins=50, range=(0, 1))
        return {
            'histogram': hist.tolist(),
            'bins': bins.tolist(),
            'cobertura': float(np.sum(mascara > 0.5) / mascara.size)
        }

    def _evaluar_calidad(self, mascara: np.ndarray) -> Dict[str, Any]:
        """Evalúa la calidad de la máscara"""
        mascara_bin = (mascara > 0.5).astype(np.uint8)

        # Conectividad
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mascara_bin)

        # Uniformidad
        uniformidad = 1.0 - np.std(mascara[mascara > 0.5]) if np.sum(mascara > 0.5) > 0 else 0

        return {
            'num_componentes': int(num_labels - 1),  # -1 para excluir fondo
            'uniformidad': float(uniformidad),
            'definicion_bordes': float(np.mean(np.abs(np.gradient(mascara)[0])))
        }

    def _analizar_fragmentacion(self, mascara: np.ndarray) -> Dict[str, Any]:
        """Analiza la fragmentación de la máscara"""
        mascara_bin = (mascara > 0.5).astype(np.uint8)
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mascara_bin)

        if num_labels <= 1:
            return {'num_fragmentos': 0, 'fragmentacion_ratio': 0.0}

        areas = stats[1:, cv2.CC_STAT_AREA]  # Excluir fondo
        area_total = np.sum(areas)
        area_mayor = np.max(areas) if len(areas) > 0 else 0

        return {
            'num_fragmentos': int(num_labels - 1),
            'fragmentacion_ratio': float(area_mayor / area_total) if area_total > 0 else 0,
            'area_fragmentos': areas.tolist()
        }

In [13]:
# =============================================================================
# GESTOR DE MÁSCARAS
# =============================================================================

class GestorMascarasYOLO:
    """Gestor especializado para almacenar/cargar máscaras YOLOv8"""

    def __init__(self, directorio_base: Path, logger: logging.Logger = None):
        self.directorio_base = directorio_base
        self.directorio_mascaras = directorio_base / "mascaras_raw"
        self.directorio_mascaras.mkdir(exist_ok=True)
        self.logger = logger

    def guardar_mascaras_imagen(self, nombre_imagen: str, mascaras_data: Dict,
                                modelo_nombre: str, config_umbral: str) -> Optional[str]:
        """Guarda máscaras en formato NPZ comprimido"""
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            nombre_base = Path(nombre_imagen).stem
            nombre_archivo = f"{modelo_nombre}_{config_umbral}_{nombre_base}_{timestamp}.npz"

            archivo_path = self.directorio_mascaras / nombre_archivo

            arrays_to_save = {}
            metadatos = {
                'nombre_imagen': nombre_imagen,
                'modelo': modelo_nombre,
                'config_umbral': config_umbral,
                'timestamp': timestamp
            }

            contador_mascaras = 0
            for umbral_key, umbral_data in mascaras_data.items():
                mascaras_raw = umbral_data.get('mascaras_raw')

                if mascaras_raw is not None and len(mascaras_raw) > 0:
                    for i, mask in enumerate(mascaras_raw):
                        if mask is not None:
                            key = f"{umbral_key}_mask_{i}"
                            arrays_to_save[key] = np.array(mask, dtype=np.float32)
                            contador_mascaras += 1

                    metadatos[f"{umbral_key}_num_mascaras"] = len(mascaras_raw)
                    metadatos[f"{umbral_key}_scores"] = umbral_data.get('confianza_scores', [])
                    metadatos[f"{umbral_key}_classes"] = umbral_data.get('classes', [])

            if contador_mascaras > 0:
                np.savez_compressed(archivo_path, metadatos=metadatos, **arrays_to_save)

                if self.logger:
                    self.logger.info(f"Máscaras guardadas: {nombre_archivo} ({contador_mascaras} máscaras)")

                return str(archivo_path.relative_to(self.directorio_base))
            else:
                if self.logger:
                    self.logger.warning(f"No hay máscaras para guardar: {nombre_imagen}")
                return None

        except Exception as e:
            if self.logger:
                self.logger.error(f"Error guardando máscaras: {str(e)}")
            return None

    def obtener_estadisticas_almacenamiento(self) -> Dict[str, Any]:
        """Estadísticas de archivos NPZ"""
        try:
            archivos_npz = list(self.directorio_mascaras.glob("*.npz"))

            if not archivos_npz:
                return {'total_archivos': 0, 'tamaño_total_mb': 0.0, 'tamaño_promedio_mb': 0.0}

            tamaños_mb = [f.stat().st_size / (1024*1024) for f in archivos_npz]

            return {
                'total_archivos': len(archivos_npz),
                'tamaño_total_mb': round(sum(tamaños_mb), 2),
                'tamaño_promedio_mb': round(np.mean(tamaños_mb), 2),
                'tamaño_min_mb': round(min(tamaños_mb), 2),
                'tamaño_max_mb': round(max(tamaños_mb), 2)
            }
        except Exception as e:
            if self.logger:
                self.logger.error(f"Error obteniendo estadísticas: {str(e)}")
            return {}

In [14]:
# =============================================================================
# DETECTOR YOLOV8
# =============================================================================

class DetectorYOLOv8:
    """Detector de personas usando YOLOv8-seg"""

    PERSON_CLASS_ID = 0  # ID de "person" en COCO

    def __init__(self, modelo_info: ModeloYOLOInfo, logger: logging.Logger,
                 extractor: ExtractorCaracteristicasAvanzado, mask_analyzer: MaskAnalyzer,
                 device: str = 'cuda'):
        self.modelo_info = modelo_info
        self.logger = logger
        self.extractor = extractor
        self.mask_analyzer = mask_analyzer
        self.device = device

        self.logger.info(f"Cargando modelo {modelo_info.nombre_modelo}...")
        self.modelo = YOLO(modelo_info.nombre_modelo)
        self.modelo.to(device)
        self.logger.info(f"Modelo {modelo_info.nombre_corto} cargado en {device}")

    def detectar_en_imagen(self, imagen: Image.Image, umbrales: List[float],
                          img_size: int = 640) -> Dict[str, Any]:
        """Detecta personas con análisis completo"""
        try:
            img_array = np.array(imagen)

            # Inferencia
            results = self.modelo.predict(
                img_array,
                imgsz=img_size,
                conf=min(umbrales),
                device=self.device,
                verbose=False
            )[0]

            boxes = results.boxes
            masks = results.masks

            if boxes is None or len(boxes) == 0:
                return {
                    'detecciones_por_umbral': {},
                    'metadata': {'total_objetos_detectados': 0, 'imagen_shape': imagen.size}
                }

            # Filtrar solo personas
            person_indices = [i for i, cls in enumerate(boxes.cls) if int(cls) == self.PERSON_CLASS_ID]

            if len(person_indices) == 0:
                return {
                    'detecciones_por_umbral': {},
                    'metadata': {'total_objetos_detectados': 0, 'personas_detectadas': 0}
                }

            person_boxes = boxes[person_indices]
            person_confs = person_boxes.conf.cpu().numpy()
            person_masks = None

            if masks is not None:
                person_masks = masks[person_indices].data.cpu().numpy()

            # Procesar por umbral
            detecciones_por_umbral = {}

            for umbral in umbrales:
                umbral_key = f"umbral_{umbral}"
                indices_validos = person_confs >= umbral
                num_detecciones = np.sum(indices_validos)

                if num_detecciones == 0:
                    detecciones_por_umbral[umbral_key] = {
                        'num_personas': 0,
                        'confianza_scores': [],
                        'classes': [],
                        'detecciones': []
                    }
                    continue

                # Extraer máscaras y características
                mascaras_validas = []
                scores_validos = person_confs[indices_validos].tolist()
                detecciones_individuales = []

                if person_masks is not None:
                    mascaras_filtradas = person_masks[indices_validos]

                    for idx, mask in enumerate(mascaras_filtradas):
                        mascaras_validas.append(mask)

                        # ANÁLISIS COMPLETO DE CADA MÁSCARA
                        caracteristicas = self.extractor.extraer_todas_caracteristicas(img_array, mask)
                        analisis_mascara = self.mask_analyzer.analizar_mascara_completa(mask, img_array)

                        deteccion_info = {
                            'id_deteccion': idx,
                            'confianza': float(scores_validos[idx]),
                            'clase': self.PERSON_CLASS_ID,
                            'caracteristicas': caracteristicas,
                            'analisis_mascara': analisis_mascara
                        }

                        detecciones_individuales.append(deteccion_info)

                detecciones_por_umbral[umbral_key] = {
                    'num_personas': int(num_detecciones),
                    'confianza_scores': scores_validos,
                    'classes': [self.PERSON_CLASS_ID] * int(num_detecciones),
                    'detecciones': detecciones_individuales,
                    '_masks_for_storage': mascaras_validas
                }

            return {
                'detecciones_por_umbral': detecciones_por_umbral,
                'metadata': {
                    'total_objetos_detectados': len(boxes),
                    'personas_detectadas': len(person_indices),
                    'imagen_shape': imagen.size,
                    'modelo': self.modelo_info.nombre_corto
                }
            }

        except Exception as e:
            self.logger.error(f"Error en detección: {str(e)}")
            import traceback
            self.logger.error(traceback.format_exc())
            return {'detecciones_por_umbral': {}, 'metadata': {'error': str(e)}}

    def liberar_memoria(self):
        """Libera memoria del modelo"""
        try:
            del self.modelo
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            self.logger.info("Memoria liberada")
        except Exception as e:
            self.logger.warning(f"Error liberando memoria: {str(e)}")

In [15]:
# =============================================================================
# SISTEMA DE LOGGING
# =============================================================================

class LoggerManager:
    """Gestor centralizado de logging"""

    def __init__(self, directorio_logs: Path):
        self.directorio_logs = directorio_logs
        self.directorio_logs.mkdir(parents=True, exist_ok=True)
        self.loggers_activos = {}
        self.timestamp_sesion = datetime.now().strftime("%Y%m%d_%H%M%S")

    def crear_logger(self, nombre: str, nivel: int = logging.INFO) -> logging.Logger:
        """Crea logger con archivo"""
        if nombre in self.loggers_activos:
            return self.loggers_activos[nombre]

        logger = logging.getLogger(nombre)
        logger.setLevel(nivel)
        logger.handlers.clear()

        archivo_log = self.directorio_logs / f"{nombre}_{self.timestamp_sesion}.log"
        file_handler = logging.FileHandler(archivo_log, encoding='utf-8')
        file_handler.setLevel(nivel)

        formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)

        self.loggers_activos[nombre] = logger
        return logger

    def obtener_estadisticas_logs(self) -> Dict[str, Any]:
        """Estadísticas de logs"""
        try:
            archivos_log = list(self.directorio_logs.glob("*.log"))
            stats = {
                'num_archivos': len(archivos_log),
                'loggers_activos': len(self.loggers_activos),
                'archivos': []
            }

            for archivo in archivos_log:
                tamaño_kb = archivo.stat().st_size / 1024
                stats['archivos'].append({
                    'nombre': archivo.name,
                    'tamaño_kb': round(tamaño_kb, 2)
                })

            return stats
        except Exception as e:
            return {'error': str(e)}

# =============================================================================
# UTILIDADES
# =============================================================================

class Utils:
    """Funciones utilitarias"""

    @staticmethod
    def cargar_imagenes(directorio: str, extensiones: Tuple = ('.jpg', '.jpeg', '.png')) -> List[str]:
        """Carga rutas de imágenes"""
        directorio_path = Path(directorio)
        if not directorio_path.exists():
            return []

        imagenes = []
        for ext in extensiones:
            imagenes.extend(directorio_path.glob(f"*{ext}"))
            imagenes.extend(directorio_path.glob(f"*{ext.upper()}"))

        return sorted([str(img) for img in imagenes])

    @staticmethod
    def cargar_imagen(ruta: str) -> Image.Image:
        """Carga imagen"""
        try:
            return Image.open(ruta).convert('RGB')
        except Exception as e:
            raise ValueError(f"Error cargando imagen {ruta}: {str(e)}")

    @staticmethod
    def guardar_json(datos: Any, archivo: Path, indent: int = 2) -> None:
        """Guarda JSON"""
        try:
            with open(archivo, 'w', encoding='utf-8') as f:
                json.dump(datos, f, indent=indent, ensure_ascii=False, default=str)
        except Exception as e:
            raise IOError(f"Error guardando JSON: {str(e)}")

In [16]:
# =============================================================================
# PROCESADOR DE RESULTADOS
# =============================================================================

class ProcesadorResultadosYOLO:
    """Procesador completo de resultados"""

    def __init__(self, directorio_base: Path, logger_manager: LoggerManager):
        self.directorio_base = directorio_base
        self.logger = logger_manager.crear_logger("procesador_yolo")
        self.gestor_mascaras = GestorMascarasYOLO(directorio_base, self.logger)

        self.imagenes_procesadas = 0
        self.imagenes_exitosas = 0
        self.tiempo_total_procesamiento = 0

    def procesar_imagen(self, ruta_imagen: str, detector: DetectorYOLOv8,
                       umbrales: List[float], modelo_nombre: str,
                       config_nombre: str) -> Optional[Dict]:
        """Procesa imagen completa con análisis"""
        try:
            inicio = time.time()

            imagen = Utils.cargar_imagen(ruta_imagen)
            nombre_archivo = Path(ruta_imagen).name

            # Detección con análisis completo
            resultados_deteccion = detector.detectar_en_imagen(imagen, umbrales)

            # Guardar máscaras
            archivo_mascaras = None
            if resultados_deteccion.get('detecciones_por_umbral'):
                mascaras_data = {}

                for umbral_key, umbral_data in resultados_deteccion['detecciones_por_umbral'].items():
                    masks_storage = umbral_data.pop('_masks_for_storage', None)

                    mascaras_data[umbral_key] = {
                        'mascaras_raw': masks_storage,
                        'confianza_scores': umbral_data.get('confianza_scores', []),
                        'classes': umbral_data.get('classes', [])
                    }

                archivo_mascaras = self.gestor_mascaras.guardar_mascaras_imagen(
                    nombre_archivo, mascaras_data, modelo_nombre, config_nombre
                )

            tiempo_ms = (time.time() - inicio) * 1000

            resultado = {
                'nombre_imagen': nombre_archivo,
                'ruta_original': ruta_imagen,
                'detecciones': resultados_deteccion['detecciones_por_umbral'],
                'metadata': resultados_deteccion.get('metadata', {}),
                'archivo_mascaras': archivo_mascaras,
                'tiempo_procesamiento_ms': round(tiempo_ms, 2),
                'librerias_usadas': ['ultralytics', 'opencv', 'shapely', 'mahotas', 'scikit-image']
            }

            self.imagenes_procesadas += 1
            self.imagenes_exitosas += 1
            self.tiempo_total_procesamiento += tiempo_ms

            return resultado

        except Exception as e:
            self.logger.error(f"Error procesando {ruta_imagen}: {str(e)}")
            self.imagenes_procesadas += 1
            return None

    def generar_resumen(self, resultados: List[Dict], config_umbral: ConfiguracionUmbrales,
                       modelo_info: ModeloYOLOInfo) -> Dict[str, Any]:
        """Genera resumen estadístico completo"""
        if not resultados:
            return {}

        resumen = {
            'modelo': {
                'nombre': modelo_info.nombre_corto,
                'tamaño': modelo_info.tamaño,
                'parametros': modelo_info.parametros,
                'tipo': 'instance_segmentation',
                'framework': 'YOLOv8'
            },
            'configuracion_umbral': {
                'nombre': config_umbral.nombre,
                'umbrales': config_umbral.valores,
                'descripcion': config_umbral.descripcion
            },
            'estadisticas_generales': {
                'total_imagenes': len(resultados),
                'imagenes_con_detecciones': 0,
                'tiempo_promedio_ms': 0,
                'tiempo_total_ms': 0
            },
            'por_umbral': {}
        }

        tiempos = []
        imagenes_con_detecciones = set()

        for resultado in resultados:
            tiempos.append(resultado.get('tiempo_procesamiento_ms', 0))

            for umbral_key, deteccion in resultado.get('detecciones', {}).items():
                if deteccion.get('num_personas', 0) > 0:
                    imagenes_con_detecciones.add(resultado['nombre_imagen'])

                    if umbral_key not in resumen['por_umbral']:
                        resumen['por_umbral'][umbral_key] = {
                            'total_personas': 0,
                            'imagenes_con_personas': 0,
                            'confianza_promedio': [],
                            'confianza_min': [],
                            'confianza_max': []
                        }

                    resumen['por_umbral'][umbral_key]['total_personas'] += deteccion['num_personas']
                    resumen['por_umbral'][umbral_key]['imagenes_con_personas'] += 1

                    scores = deteccion.get('confianza_scores', [])
                    if scores:
                        resumen['por_umbral'][umbral_key]['confianza_promedio'].extend(scores)
                        resumen['por_umbral'][umbral_key]['confianza_min'].append(min(scores))
                        resumen['por_umbral'][umbral_key]['confianza_max'].append(max(scores))

        resumen['estadisticas_generales']['imagenes_con_detecciones'] = len(imagenes_con_detecciones)
        resumen['estadisticas_generales']['tiempo_promedio_ms'] = round(np.mean(tiempos), 2) if tiempos else 0
        resumen['estadisticas_generales']['tiempo_total_ms'] = round(sum(tiempos), 2)

        # Promediar estadísticas de confianza
        for umbral_key in resumen['por_umbral']:
            confs = resumen['por_umbral'][umbral_key]['confianza_promedio']
            mins = resumen['por_umbral'][umbral_key]['confianza_min']
            maxs = resumen['por_umbral'][umbral_key]['confianza_max']

            resumen['por_umbral'][umbral_key]['confianza_promedio'] = round(np.mean(confs), 3) if confs else 0
            resumen['por_umbral'][umbral_key]['confianza_min'] = round(min(mins), 3) if mins else 0
            resumen['por_umbral'][umbral_key]['confianza_max'] = round(max(maxs), 3) if maxs else 0

        return resumen

    def mostrar_resumen_consola(self, resumen: Dict):
        """Muestra resumen en consola"""
        print(f"\n{'='*80}")
        print(f"RESUMEN ESTADÍSTICO")
        print(f"{'='*80}")

        if 'estadisticas_generales' in resumen:
            stats = resumen['estadisticas_generales']
            print(f"Total imágenes: {stats['total_imagenes']}")
            print(f"Imágenes con detecciones: {stats['imagenes_con_detecciones']}")
            print(f"Tiempo promedio: {stats['tiempo_promedio_ms']:.2f}ms")

        if 'por_umbral' in resumen:
            print(f"\nDETECCIONES POR UMBRAL:")
            for umbral, datos in resumen['por_umbral'].items():
                print(f"  {umbral}:")
                print(f"    - Personas detectadas: {datos['total_personas']}")
                print(f"    - Imágenes con personas: {datos['imagenes_con_personas']}")
                print(f"    - Confianza promedio: {datos['confianza_promedio']:.3f}")


In [17]:
# =============================================================================
# EVALUADOR PRINCIPAL
# =============================================================================

class EvaluadorYOLOv8:
    """Evaluador principal para YOLOv8-seg"""

    def __init__(self, config: ConfigEvaluacionYOLO):
        self.config = config
        self.directorio_ejecucion = config.crear_directorio_ejecucion()
        print(f"\n📁 Directorio de ejecución: {self.directorio_ejecucion}")

        self.logger_manager = LoggerManager(self.directorio_ejecucion / "logs")
        self.logger = self.logger_manager.crear_logger("evaluador_principal")
        self.procesador = ProcesadorResultadosYOLO(self.directorio_ejecucion, self.logger_manager)

        self.logger.info("Evaluador YOLOv8 inicializado")

    def ejecutar_evaluacion_modelo(self, idx_modelo: int, nombre_config_umbral: str) -> Optional[str]:
        """Ejecuta evaluación completa"""
        if idx_modelo >= len(self.config.MODELOS):
            self.logger.error(f"Índice de modelo inválido: {idx_modelo}")
            return None

        if nombre_config_umbral not in self.config.UMBRALES:
            self.logger.error(f"Configuración de umbral inválida: {nombre_config_umbral}")
            return None

        modelo_info = self.config.MODELOS[idx_modelo]
        config_umbral = self.config.UMBRALES[nombre_config_umbral]

        self.logger.info(f"Iniciando: {modelo_info.nombre_corto} - {nombre_config_umbral}")

        try:
            imagenes = Utils.cargar_imagenes(str(self.config.DATASET_PATH))
            if not imagenes:
                self.logger.error("No se encontraron imágenes")
                return None

            self.logger.info(f"Dataset: {len(imagenes)} imágenes")

            if len(imagenes) > self.config.MAX_IMAGENES_LOTE:
                imagenes = imagenes[:self.config.MAX_IMAGENES_LOTE]
                self.logger.info(f"Limitando a {self.config.MAX_IMAGENES_LOTE} imágenes")

            # Inicializar detector con componentes avanzados
            detector_logger = self.logger_manager.crear_logger(f"detector_{modelo_info.nombre_corto}")
            extractor = ExtractorCaracteristicasAvanzado(detector_logger)
            mask_analyzer = MaskAnalyzer(detector_logger)

            detector = DetectorYOLOv8(
                modelo_info, detector_logger, extractor, mask_analyzer, self.config.DEVICE
            )

            # Nombres
            if self.config.USAR_NOMBRES_SANITIZADOS:
                modelo_nombre = modelo_info.obtener_nombre_sanitizado()
                config_nombre = config_umbral.obtener_nombre_sanitizado()
            else:
                modelo_nombre = modelo_info.nombre_corto
                config_nombre = config_umbral.nombre

            # Procesar
            resultados = []
            tiempo_inicio = time.time()

            for i, ruta_imagen in enumerate(tqdm(imagenes, desc=f"Procesando {modelo_info.nombre_corto}"), 1):
                if i % self.config.LIMPIAR_CACHE_CADA == 0:
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    self.logger.info(f"Caché limpiado en imagen {i}")

                resultado = self.procesador.procesar_imagen(
                    ruta_imagen, detector, config_umbral.valores,
                    modelo_nombre, config_nombre
                )

                if resultado:
                    resultados.append(resultado)

            tiempo_total = time.time() - tiempo_inicio
            self.logger.info(f"Completado en {tiempo_total:.1f}s")

            # Resumen
            resumen = self.procesador.generar_resumen(resultados, config_umbral, modelo_info)
            self.procesador.mostrar_resumen_consola(resumen)

            # Guardar
            archivo_generado = self._guardar_resultados(
                resultados, resumen, modelo_info, config_umbral,
                modelo_nombre, config_nombre
            )

            # Estadísticas de máscaras
            stats_mascaras = self.procesador.gestor_mascaras.obtener_estadisticas_almacenamiento()
            if stats_mascaras['total_archivos'] > 0:
                print(f"\nESTADÍSTICAS DE MÁSCARAS:")
                print(f"   Archivos NPZ: {stats_mascaras['total_archivos']}")
                print(f"   Tamaño total: {stats_mascaras['tamaño_total_mb']} MB")

            detector.liberar_memoria()

            return archivo_generado

        except Exception as e:
            self.logger.error(f"Error en evaluación: {str(e)}")
            import traceback
            self.logger.error(traceback.format_exc())
            return None

    def _guardar_resultados(self, resultados: List[Dict], resumen: Dict,
                           modelo_info: ModeloYOLOInfo, config_umbral: ConfiguracionUmbrales,
                           modelo_nombre: str, config_nombre: str) -> Optional[str]:
        """Guarda resultados completos en JSON"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        nombre_archivo = f"{modelo_nombre}_{config_nombre}_{timestamp}.json"

        try:
            directorio_modelo = self.directorio_ejecucion / modelo_nombre
            directorio_modelo.mkdir(exist_ok=True)

            archivo_principal = directorio_modelo / nombre_archivo

            datos_completos = {
                'metadata': {
                    'timestamp': datetime.now().isoformat(),
                    'version_framework': '1.0_yolov8_complete',
                    'modelo': asdict(modelo_info),
                    'configuracion_umbral': asdict(config_umbral),
                    'librerias_usadas': [
                        'ultralytics', 'opencv', 'shapely', 'mahotas',
                        'scikit-image', 'scipy', 'numpy', 'torch'
                    ]
                },
                'resumen': resumen,
                'resultados': resultados
            }

            Utils.guardar_json(datos_completos, archivo_principal)
            self.logger.info(f"Resultados guardados: {archivo_principal}")

            return str(archivo_principal)

        except Exception as e:
            self.logger.error(f"Error guardando resultados: {str(e)}")
            return None

    def ejecutar_evaluacion_completa(self) -> Dict[str, List[str]]:
        """Ejecuta evaluación de todos los modelos"""
        total_combinaciones = len(self.config.MODELOS) * len(self.config.UMBRALES)
        archivos_generados = {}

        print(f"\n{'='*100}")
        print(f"EVALUACIÓN COMPLETA YOLOV8-SEG")
        print(f"Total combinaciones: {total_combinaciones}")
        print(f"Análisis completo: Geometría, Textura, Color, Forma, Complejidad")
        print(f"{'='*100}")

        self.logger.info(f"Iniciando evaluación: {total_combinaciones} combinaciones")

        combinacion_actual = 0

        for idx_modelo, modelo_info in enumerate(self.config.MODELOS):
            archivos_modelo = []

            for nombre_config in self.config.UMBRALES.keys():
                combinacion_actual += 1

                print(f"\n{'='*100}")
                print(f"COMBINACIÓN {combinacion_actual}/{total_combinaciones}")
                print(f"Modelo: {modelo_info.nombre_corto}")
                print(f"Configuración: {nombre_config}")
                print(f"{'='*100}")

                archivo = self.ejecutar_evaluacion_modelo(idx_modelo, nombre_config)

                if archivo:
                    archivos_modelo.append(archivo)
                    print(f"✅ Evaluación completada")
                else:
                    print(f"❌ Error en evaluación")

                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

            if archivos_modelo:
                archivos_generados[modelo_info.nombre_corto] = archivos_modelo

        return archivos_generados

    def obtener_estadisticas_ejecucion(self) -> Dict[str, Any]:
        """Estadísticas finales"""
        try:
            stats_logs = self.logger_manager.obtener_estadisticas_logs()
            stats_mascaras = self.procesador.gestor_mascaras.obtener_estadisticas_almacenamiento()

            archivos_por_modelo = {}
            for modelo_dir in self.directorio_ejecucion.iterdir():
                if modelo_dir.is_dir() and modelo_dir.name not in ["logs", "visualizaciones", "formato_coco", "mascaras_raw"]:
                    archivos_json = list(modelo_dir.glob("*.json"))
                    archivos_por_modelo[modelo_dir.name] = len(archivos_json)

            return {
                'directorio_ejecucion': str(self.directorio_ejecucion),
                'modelos_procesados': len(archivos_por_modelo),
                'archivos_por_modelo': archivos_por_modelo,
                'total_archivos_json': sum(archivos_por_modelo.values()),
                'estadisticas_logs': stats_logs,
                'estadisticas_mascaras': stats_mascaras,
                'imagenes_procesadas': self.procesador.imagenes_procesadas,
                'imagenes_exitosas': self.procesador.imagenes_exitosas,
                'tiempo_total_procesamiento_ms': self.procesador.tiempo_total_procesamiento
            }
        except Exception as e:
            self.logger.error(f"Error obteniendo estadísticas: {str(e)}")
            return {}

In [18]:
# =============================================================================
# FUNCIÓN PRINCIPAL
# =============================================================================

def main():
    """Función principal"""
    try:
        config = ConfigEvaluacionYOLO()

        print(f"\n{'='*100}")
        print(f"SISTEMA DE EVALUACIÓN YOLOV8-SEGMENTATION")
        print(f"{'='*100}")

        print(f"\nModelos disponibles: {len(config.MODELOS)}")
        for i, modelo in enumerate(config.MODELOS):
            print(f"   [{i}] {modelo.nombre_corto} ({modelo.tamaño}, {modelo.parametros})")

        print(f"\nConfiguraciones de umbral: {len(config.UMBRALES)}")
        for nombre, cfg in config.UMBRALES.items():
            print(f"   {nombre}: {cfg.descripcion}")

        if not config.validar_configuracion():
            print("❌ ERROR: Configuración inválida")
            return

        print("✅ Configuración validada")

        evaluador = EvaluadorYOLOv8(config)

        resumen_config = config.obtener_resumen_configuracion()
        print(f"\nRESUMEN:")
        print(f"   Total combinaciones: {resumen_config['total_combinaciones']}")
        print(f"   Dataset: {resumen_config['dataset_path']}")
        print(f"   Device: {resumen_config['device']}")
        print(f"   Image size: {resumen_config['img_size']}")

        print(f"\n🚀 INICIANDO EVALUACIÓN AUTOMÁTICA COMPLETA...")
        archivos_resultado = evaluador.ejecutar_evaluacion_completa()

        # Estadísticas finales
        estadisticas = evaluador.obtener_estadisticas_ejecucion()

        # Resumen final
        print(f"\n{'='*100}")
        print(f"✅ EVALUACIÓN COMPLETA FINALIZADA EXITOSAMENTE")
        print(f"{'='*100}")

        if estadisticas:
            print(f"📁 Directorio: {estadisticas['directorio_ejecucion']}")
            print(f"📊 Imágenes procesadas: {estadisticas['imagenes_procesadas']}")
            print(f"✅ Imágenes exitosas: {estadisticas['imagenes_exitosas']}")
            print(f"🤖 Modelos evaluados: {estadisticas['modelos_procesados']}")
            print(f"📄 Archivos JSON: {estadisticas['total_archivos_json']}")

            if estadisticas.get('tiempo_total_procesamiento_ms'):
                tiempo_min = estadisticas['tiempo_total_procesamiento_ms'] / 60000
                print(f"⏱️ Tiempo total: {tiempo_min:.1f} minutos")

            if estadisticas.get('estadisticas_mascaras'):
                stats_m = estadisticas['estadisticas_mascaras']
                print(f"\n💾 ESTADÍSTICAS DE MÁSCARAS:")
                print(f"   Total archivos NPZ: {stats_m['total_archivos']}")
                print(f"   Tamaño total: {stats_m['tamaño_total_mb']} MB")
                print(f"   Tamaño promedio: {stats_m['tamaño_promedio_mb']} MB")

            print(f"\n📂 RESUMEN POR MODELO:")
            for modelo, num_archivos in estadisticas['archivos_por_modelo'].items():
                print(f"   {modelo}: {num_archivos} evaluaciones completadas")

            if estadisticas.get('estadisticas_logs'):
                logs = estadisticas['estadisticas_logs']
                print(f"\n📝 LOGS GENERADOS:")
                print(f"   Archivos de log: {logs['num_archivos']}")
                for archivo in logs.get('archivos', []):
                    print(f"      - {archivo['nombre']}: {archivo['tamaño_kb']:.2f} KB")

        print(f"\n✅ ANÁLISIS COMPLETADO CON ÉXITO")

    except Exception as e:
        print(f"\n❌ ERROR FATAL: {str(e)}")
        import traceback
        traceback.print_exc()

In [23]:
# =============================================================================
# EJECUCIÓN
# =============================================================================

if __name__ == "__main__":
    main()


SISTEMA DE EVALUACIÓN YOLOV8-SEGMENTATION

Modelos disponibles: 5
   [0] yolov8n-seg (nano, 3.4M)
   [1] yolov8s-seg (small, 11.8M)
   [2] yolov8m-seg (medium, 27.3M)
   [3] yolov8l-seg (large, 46.0M)
   [4] yolov8x-seg (xlarge, 71.8M)

Configuraciones de umbral: 4
   ultra_sensible: Máxima sensibilidad
   alta_sensibilidad: Alta sensibilidad
   balanceada: Balance
   conservadora: Conservadora


INFO:evaluador_principal:Evaluador YOLOv8 inicializado
INFO:evaluador_principal:Iniciando evaluación: 20 combinaciones
INFO:evaluador_principal:Iniciando: yolov8n-seg - ultra_sensible


✅ Configuración validada

📁 Directorio de ejecución: /content/drive/MyDrive/TFM/yolov8_seg/resultados/ejecucion_20251009_192747

RESUMEN:
   Total combinaciones: 20
   Dataset: /content/drive/MyDrive/TFM/yolov8_seg/imagenes
   Device: cpu
   Image size: 640

🚀 INICIANDO EVALUACIÓN AUTOMÁTICA COMPLETA...

EVALUACIÓN COMPLETA YOLOV8-SEG
Total combinaciones: 20
Análisis completo: Geometría, Textura, Color, Forma, Complejidad

COMBINACIÓN 1/20
Modelo: yolov8n-seg
Configuración: ultra_sensible


INFO:evaluador_principal:Dataset: 6 imágenes
INFO:detector_yolov8n-seg:Cargando modelo yolov8n-seg.pt...


INFO:detector_yolov8n-seg:Modelo yolov8n-seg cargado en cpu
Procesando yolov8n-seg: 100%|██████████| 6/6 [00:12<00:00,  2.01s/it]
INFO:evaluador_principal:Completado en 12.0s
INFO:evaluador_principal:Resultados guardados: /content/drive/MyDrive/TFM/yolov8_seg/resultados/ejecucion_20251009_192747/yv8n_seg/yv8n_seg_cfg_ultr_20251009_192800.json
INFO:detector_yolov8n-seg:Memoria liberada
INFO:evaluador_principal:Iniciando: yolov8n-seg - alta_sensibilidad
INFO:evaluador_principal:Dataset: 6 imágenes
INFO:detector_yolov8n-seg:Cargando modelo yolov8n-seg.pt...
INFO:detector_yolov8n-seg:Modelo yolov8n-seg cargado en cpu


...


RESUMEN ESTADÍSTICO
Total imágenes: 6
Imágenes con detecciones: 6
Tiempo promedio: 3680.93ms

DETECCIONES POR UMBRAL:
  umbral_0.1:
    - Personas detectadas: 6
    - Imágenes con personas: 6
    - Confianza promedio: 0.944
  umbral_0.2:
    - Personas detectadas: 6
    - Imágenes con personas: 6
    - Confianza promedio: 0.944
  umbral_0.3:
    - Personas detectadas: 6
    - Imágenes con personas: 6
    - Confianza promedio: 0.944
  umbral_0.4:
    - Personas detectadas: 6
    - Imágenes con personas: 6
    - Confianza promedio: 0.944

ESTADÍSTICAS DE MÁSCARAS:
   Archivos NPZ: 108
   Tamaño total: 1.63 MB
✅ Evaluación completada

COMBINACIÓN 19/20
Modelo: yolov8x-seg
Configuración: balanceada


INFO:detector_yolov8x-seg:Modelo yolov8x-seg cargado en cpu
Procesando yolov8x-seg: 100%|██████████| 6/6 [00:21<00:00,  3.64s/it]
INFO:evaluador_principal:Completado en 21.8s
INFO:evaluador_principal:Resultados guardados: /content/drive/MyDrive/TFM/yolov8_seg/resultados/ejecucion_20251009_192747/yv8x_seg/yv8x_seg_cfg_bala_20251009_193136.json
INFO:detector_yolov8x-seg:Memoria liberada
INFO:evaluador_principal:Iniciando: yolov8x-seg - conservadora
INFO:evaluador_principal:Dataset: 6 imágenes
INFO:detector_yolov8x-seg:Cargando modelo yolov8x-seg.pt...



RESUMEN ESTADÍSTICO
Total imágenes: 6
Imágenes con detecciones: 6
Tiempo promedio: 3633.43ms

DETECCIONES POR UMBRAL:
  umbral_0.25:
    - Personas detectadas: 6
    - Imágenes con personas: 6
    - Confianza promedio: 0.944
  umbral_0.5:
    - Personas detectadas: 6
    - Imágenes con personas: 6
    - Confianza promedio: 0.944
  umbral_0.7:
    - Personas detectadas: 6
    - Imágenes con personas: 6
    - Confianza promedio: 0.944

ESTADÍSTICAS DE MÁSCARAS:
   Archivos NPZ: 114
   Tamaño total: 1.7 MB
✅ Evaluación completada

COMBINACIÓN 20/20
Modelo: yolov8x-seg
Configuración: conservadora


INFO:detector_yolov8x-seg:Modelo yolov8x-seg cargado en cpu
Procesando yolov8x-seg: 100%|██████████| 6/6 [00:22<00:00,  3.67s/it]
INFO:evaluador_principal:Completado en 22.0s
INFO:evaluador_principal:Resultados guardados: /content/drive/MyDrive/TFM/yolov8_seg/resultados/ejecucion_20251009_192747/yv8x_seg/yv8x_seg_cfg_cons_20251009_193159.json
INFO:detector_yolov8x-seg:Memoria liberada



RESUMEN ESTADÍSTICO
Total imágenes: 6
Imágenes con detecciones: 6
Tiempo promedio: 3661.13ms

DETECCIONES POR UMBRAL:
  umbral_0.5:
    - Personas detectadas: 6
    - Imágenes con personas: 6
    - Confianza promedio: 0.944
  umbral_0.7:
    - Personas detectadas: 6
    - Imágenes con personas: 6
    - Confianza promedio: 0.944
  umbral_0.85:
    - Personas detectadas: 6
    - Imágenes con personas: 6
    - Confianza promedio: 0.944

ESTADÍSTICAS DE MÁSCARAS:
   Archivos NPZ: 120
   Tamaño total: 1.77 MB
✅ Evaluación completada

✅ EVALUACIÓN COMPLETA FINALIZADA EXITOSAMENTE
📁 Directorio: /content/drive/MyDrive/TFM/yolov8_seg/resultados/ejecucion_20251009_192747
📊 Imágenes procesadas: 120
✅ Imágenes exitosas: 120
🤖 Modelos evaluados: 5
📄 Archivos JSON: 20
⏱️ Tiempo total: 3.9 minutos

💾 ESTADÍSTICAS DE MÁSCARAS:
   Total archivos NPZ: 120
   Tamaño total: 1.77 MB
   Tamaño promedio: 0.01 MB

📂 RESUMEN POR MODELO:
   yv8n_seg: 4 evaluaciones completadas
   yv8s_seg: 4 evaluaciones compl